In [1]:
pip install pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
transacciones = pd.read_csv("datos/base_transacciones_final.csv", parse_dates=["fecha"])
clientes = pd.read_csv("datos/base_clientes_final.csv", parse_dates=["fecha_nacimiento", "fecha_alta"])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

=== 2. Renombrar id a id_cliente para merge
transacciones.rename(columns={"id": "id_cliente"}, inplace=True)
clientes.rename(columns={"id": "id_cliente"}, inplace=True)

# === 3. Unir las bases ===
df = transacciones.merge(clientes, on="id_cliente", how="left")

# === 4. Ordenar por cliente y fecha ===
df = df.sort_values(by=["id_cliente", "fecha"])

# === 5. Calcular días hasta próxima transacción ===
df["fecha_siguiente"] = df.groupby("id_cliente")["fecha"].shift(-1)
df["dias_hasta_proximo"] = (df["fecha_siguiente"] - df["fecha"]).dt.days
df = df.dropna(subset=["dias_hasta_proximo"])

# === 6. Features adicionales ===
df["edad"] = ((df["fecha"] - df["fecha_nacimiento"]).dt.days / 365).astype(int)
df["antiguedad"] = (df["fecha"] - df["fecha_alta"]).dt.days
df["mes"] = df["fecha"].dt.month
df["dia_semana"] = df["fecha"].dt.weekday
df["dias_desde_ultimo"] = df.groupby("id_cliente")["fecha"].diff().dt.days
df["dias_desde_ultimo"] = df["dias_desde_ultimo"].fillna(df["dias_desde_ultimo"].median())

# === 7. Variables y target ===
X = df[[
    "mes", "dia_semana", "edad", "antiguedad", "dias_desde_ultimo",
    "giro_comercio", "tipo_venta", "genero", "actividad_empresarial", "tipo_persona"
]]
y = df["dias_hasta_proximo"]

# === 8. Preprocesamiento ===
numeric_features = ["mes", "dia_semana", "edad", "antiguedad", "dias_desde_ultimo"]
categorical_features = ["giro_comercio", "tipo_venta", "genero", "actividad_empresarial", "tipo_persona"]

preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# === 9. Pipeline y modelo ===
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42))
])

# === 10. Entrenamiento ===
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# === 11. Evaluación ===
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE (Error absoluto medio): {mae:.2f} días")


In [11]:
pip install streamlit matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.8 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 36.4 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 13.1 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# app.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# --- Simula datos (puedes reemplazar esto con tus datos reales) ---
np.random.seed(42)
fechas = pd.date_range("2024-01-01", "2024-12-31")
clientes = ['Cliente1']
categorias = ['Comida', 'Transporte', 'Entretenimiento', 'Servicios', 'Ropa']

data = []
for cliente in clientes:
    for _ in range(200):
        fecha = np.random.choice(fechas)
        categoria = np.random.choice(categorias)
        monto = np.random.randint(50, 1000)
        data.append([cliente, fecha, categoria, monto])

df = pd.DataFrame(data, columns=["cliente_id", "fecha", "categoria", "monto"])

# --- Preprocesamiento ---
df["mes"] = df["fecha"].dt.month_name()
df["año"] = df["fecha"].dt.year

gasto_mensual = df.groupby("mes")["monto"].sum().sort_values(ascending=False)
categoria_fav = df.groupby("categoria")["monto"].sum().idxmax()
mes_top = gasto_mensual.idxmax()
monto_mes_top = gasto_mensual.max()

# --- Análisis de ahorro ---
ultimo_mes = df[df["fecha"] > "2024-11-01"]
promedio_ultimo = ultimo_mes["monto"].mean()
promedio_anual = df["monto"].mean()
modo_ahorro = promedio_ultimo < promedio_anual * 0.8

top_categorias = df["categoria"].value_counts().head(2).index.tolist()
sugerencias = {
    "Comida": "🍳 Prepara comida en casa 3 días a la semana para ahorrar $500.",
    "Transporte": "🚇 Cambia 2 viajes en Uber por transporte público, ahorras $300.",
    "Entretenimiento": "📺 Cancela suscripciones que no usaste este mes.",
}
recomendaciones = [sugerencias[c] for c in top_categorias if c in sugerencias]

# --- Streamlit Layout ---
st.set_page_config(page_title="Wrapped de Gastos", layout="wide")
st.title("🎁 Tu Wrapped de Gastos 2024")

st.subheader("Resumen del año")
st.markdown(f"""
- Tu mes más costoso fue **{mes_top}** con un total de **${monto_mes_top:.2f}**.
- Gastaste más en la categoría **{categoria_fav}**.
- Realizaste **{len(df)}** transacciones este año.
- Tu gasto promedio fue de **${df['monto'].mean():.2f}**.
""")

st.divider()

# Visualizaciones
col1, col2 = st.columns(2)

with col1:
    st.subheader("📊 Gasto por Mes")
    fig, ax = plt.subplots()
    gasto_mensual.plot(kind="bar", ax=ax)
    ax.set_ylabel("Monto ($)")
    ax.set_title("Gasto por Mes")
    st.pyplot(fig)

with col2:
    st.subheader("🥧 Distribución por Categoría")
    fig2, ax2 = plt.subplots()
    df.groupby("categoria")["monto"].sum().plot(kind="pie", autopct='%1.1f%%', ax=ax2)
    ax2.set_ylabel("")
    ax2.set_title("Gasto por Categoría")
    st.pyplot(fig2)

st.divider()

# Modo Ahorro
st.subheader("💰 Modo Ahorro Inteligente")

if modo_ahorro:
    st.success("¡Modo Ahorro Activado!")
    for r in recomendaciones:
        st.markdown(f"- {r}")
else:
    st.info("No se detectó comportamiento de ahorro significativo este mes.")

st.caption("Generado automáticamente por tu asistente 🧠")

